In [ ]:
import numpy as np                                                                                                                 
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

In [ ]:
#Define functions used for diffusion partial differential equations

#Define Laplace operator with zero-flux boundary condition
def dd2 (A,x,y,dx,dy,i,j):
  if i==0:
    dAx2=(A[i+1,j]-2*A[i,j]+A[i+1,j])/(dx*dx)
  elif i==x-1:
    dAx2=(A[i-1,j]-2*A[i,j]+A[i-1,j])/(dx*dx)
  else:
    dAx2=(A[i+1,j]-2*A[i,j]+A[i-1,j])/(dx*dx)

  if j==0:
    dAy2=(A[i,j+1]-2*A[i,j]+A[i,j+1])/(dy*dy)
  elif j==y-1:
    dAy2=(A[i,j-1]-2*A[i,j]+A[i,j-1])/(dy*dy)
  else:
    dAy2=(A[i,j+1]-2*A[i,j]+A[i,j-1])/(dy*dy)
    
  d=dAx2+dAy2

  return d;

#Define Erk propagation function
#E0, initial condition
#Cell id, cells position
#T, total simulation time
#dt, simulation time step
#c_threhold, activition threshold
#cell_len, cell length
#activition rate, a
#Diffusion coefficient, D

def Erk_Propogation_2D (E0,Cell_id,num_grid_x,num_grid_y,grid_size,T,dt,step,c_threhold,cell_len,a):  
   t1=0
   t2=0
   E=E0.copy()
   v=0
   L=np.zeros((step-1,num_grid_x))
   for time in range(step-1): 
     Es=E
     for i in range(num_grid_x):
       for j in range(num_grid_y):
         if i<10:
           E[i,j]=Es[i,j]+dt*(D*dd2(Es,num_grid_x,num_grid_y,grid_size,grid_size,i,j)+a*(1-Es[i,j]))
         else:
           E[i,j]=Es[i,j]+dt*(D*dd2(Es,num_grid_x,num_grid_y,grid_size,grid_size,i,j)+a*theta_n(Es,Cell_id,i,j,c_threhold,cell_len))
     if t1==0:
       if np.min(E[11,:])>0.1:
         t1=time
     if t2==0:
       if np.min(E[num_grid_x-5,:])>0.1:
         t2=time
         v=((num_grid_x-16)*cell_len*grid_size)/((t2-t1)*dt)
     else:
        break
     L[time,:]=np.mean(E, axis=1).T
       
   return E,v,L;

#Define Heaviside step function
def theta_n (E,Cell_id,i,j,c_threhold,cell_len): 
  c=0
  if Cell_id[i,j]<0:
    print('cell identity error')    
  if Cell_id[i,j]==0:
    c=0
  else:
    if max(E[int(i-Cell_id[i,j]+1):min(int(i-Cell_id[i,j]+1+cell_len),num_grid_x),j])>c_threhold:
        c=1-E[i,j]
  return c;

#Generate cell pattern for signal propagation with given parameters
#num_grid_x, num_grid_y, number of cells on x- and y-axis
#bias, distribution bias of cells
#den_volum, volume density of long cells
#cell_len, long cell length

def Generate_cell_id (num_grid_x,num_grid_y,bias,den_volum,cell_len): 
  Cell_id=np.zeros((num_grid_x,num_grid_y))-1
  dev_h=den_volum+bias
  dev_l=den_volum-bias
  density_h=dev_h/(cell_len+dev_h-cell_len*dev_h)
  density_l=dev_l/(cell_len+dev_l-cell_len*dev_l)
  for j in range(num_grid_y):
    i=0
    while i<num_grid_x:
      indic=np.random.uniform()
      if j<0.5*num_grid_y:
        den= density_h
      else:
        den= density_l
      if indic>den:
        Cell_id[i,j]=0
        i=i+1
      else:
        d=min(cell_len,num_grid_x-i)
        Cell_id[i:i+d,j]=np.linspace(1,d,num=d)
        i=i+cell_len

  return Cell_id;

In [ ]:
#Find all cells in the same cluster of cell at position i,j
def Find_all_neighber (Cell_id,i,j):
    Index = np.zeros((1,2))
    Search = np.zeros((1,1))+1
    Index[0,0]=i
    Index[0,1]=j
    m,n=np.shape(Cell_id)
    Havein = np.zeros((m,n))
    Havein[i,j]=1
    while np.sum(Search)>0:
        for i in range (len(Search)):
            if Search[i]>0:
                x=min(int(Index[i,0]+1),m-1)
                y=int(Index[i,1])
                if (Cell_id[x,y]>0) and (Havein[x,y]<1):
                    Index=np.vstack((Index,[x,y]))
                    Search=np.append(Search,1)
                    Havein[x,y]=1
                
                x=int(Index[i,0])
                y=min(int(Index[i,1]+1),n-1)
                if (Cell_id[x,y]>0) and (Havein[x,y]<1):
                    Index=np.vstack((Index,[x,y]))
                    Search=np.append(Search,1)
                    Havein[x,y]=1
                
                x=max(int(Index[i,0]-1),0)
                y=int(Index[i,1])
                if (Cell_id[x,y]>0) and (Havein[x,y]<1):
                    Index=np.vstack((Index,[x,y]))
                    Search=np.append(Search,1)
                    Havein[x,y]=1
                
                x=int(Index[i,0])
                y=max(int(Index[i,1]-1),0)
                if (Cell_id[x,y]>0) and (Havein[x,y]<1):
                    Index=np.vstack((Index,[x,y]))
                    Search=np.append(Search,1)
                    Havein[x,y]=1
                
                Search[i]=0
            
    return Index, Havein

#Calculater largest size of cell cluster for cell pattern Cell_id
def Find_most_distance (Cell_id):
    m,n=np.shape(Cell_id)
    Havein_sum=np.zeros((m,n))
    dism=0
    for i in range(10):
        for j in range(n):
            if Havein_sum[i,j]==0:
                Index, Havein=Find_all_neighber (Cell_id,i,j)
                Havein_sum=Havein_sum+Havein
                dis=np.max(Index[:,0])
                dism=max(dism,dis)
    
    return dism

#Extract data for plotting
def data_plot_clen_v(den,v):
    #0den #1len #2dis $3speed
    xdata=np.zeros(0) #den
    ydata=np.zeros(0) #len
    z1data=np.zeros(0) #dis
    z2data=np.zeros(0) #speed
    
    c_len=np.linspace(1,10,10) #cell_len
    speed=np.zeros((5,len(c_len)))
    
    for i in range(len(v[0,:])):
        if v[0,i]==den:
            xdata=np.append(xdata,v[0,i])
            ydata=np.append(ydata,v[1,i])
            z1data=np.append(z1data,v[2,i])
            z2data=np.append(z2data,60*v[3,i]/v[1,i])
    
    for m in range(5):
        for n in range(10):
            speed[m,n]=z2data[m*10+n]
    
    return xdata, ydata, z1data, z2data, speed

def data_plot_n(c_len,v):
    #0den #1len #2dis $3speed
    xdata=np.zeros(0) #den
    ydata=np.zeros(0) #len
    z1data=np.zeros(0) #dis
    z2data=np.zeros(0) #speed
    
    den_space=np.linspace(0,1,21) #cell_len
    speed=np.zeros((5,len(den_space)))
    
    for i in range(len(v[0,:])):
        if v[1,i]==c_len:
            xdata=np.append(xdata,v[0,i])
            ydata=np.append(ydata,v[1,i])
            z1data=np.append(z1data,v[2,i])
            z2data=np.append(z2data,60*v[3,i]/v[1,i])
    
    for m in range(5):
        for n in range(len(den_space)):
            speed[m,n]=z2data[m*len(den_space)+n]
    
    return xdata, ydata, z1data, z2data, speed

def data_plot_clus(c_len,v):
    #0den #1len #2dis $3speed
    xdata=np.zeros(0) #den
    ydata=np.zeros(0) #len
    z1data=np.zeros(0) #dis
    z2data=np.zeros(0) #speed
    
    den_space=np.linspace(0,1,21) #cell_len
    cluster=np.zeros((5,len(den_space)))
    
    for i in range(len(v[0,:])):
        if v[1,i]==c_len:
            xdata=np.append(xdata,v[0,i])
            ydata=np.append(ydata,v[1,i])
            z1data=np.append(z1data,v[2,i])
            z2data=np.append(z2data,v[3,i])
    
    for m in range(5):
        for n in range(len(den_space)):
            cluster[m,n]=z1data[m*len(den_space)+n]
    
    return xdata, ydata, z1data, z2data, cluster

In [ ]:
#parameter set up

T=90 #min

range_grid_x=1000
range_grid_y=200
grid_size=10
num_grid_x=int(range_grid_x/grid_size)
num_grid_y=int(range_grid_y/grid_size)
E0=np.zeros((num_grid_x,num_grid_y))

c_threhold=0.1
D=0.1*60
a=0.2
dt=0.01
step=int(T/dt)

cell_len=1
den_volum=1

bias=0

In [ ]:
#Single simulation
E9,v,L2 = Erk_Propogation_2D (E0,Cell_id,num_grid_x,num_grid_y,grid_size,T,dt,step,c_threhold,cell_len,a)

In [ ]:
#wave front plot

plt.figure(figsize=(4, 3))

plt.plot(x[0:40],E9[0:40,10],color=c1,alpha=0.7)

In [ ]:
#cell pattern plot

plt.imshow(Cell_plot, cmap='tab10', interpolation='nearest')
plt.savefig('cell_id_c10_d05.pdf')

In [ ]:
#2D signal pattern plot

plt.imshow(E, interpolation='nearest')
plt.colorbar()
plt.savefig('E_c11_d1.pdf')

In [ ]:
#parameters set up for testing varying long cell density of length

T=500 #min

range_grid_x=1000
range_grid_y=200
grid_size=10
num_grid_x=int(range_grid_x/grid_size)
num_grid_y=int(range_grid_y/grid_size)
E0=np.zeros((num_grid_x,num_grid_y))

c_threhold=0.1
D=0.1*60
a=0.2
dt=0.01
step=int(T/dt)

den_space=np.linspace(0,1,21)
print(den_space)

c_len=np.linspace(1,10,10)
c_len=c_len.astype(int)
print(c_len)

bias=0

In [ ]:
#simulation
repeat=5
data=np.zeros((4,repeat*len(den_space)*len(c_len))) #0 den, 1 len, 2 dis, 3 speed
for r in range(repeat):
    print(r)
    for i in range(len(den_space)):
        for j in range(len(c_len)):
            E0=np.zeros((num_grid_x,num_grid_y))
            Cell_id=Generate_cell_id (num_grid_x,num_grid_y,bias,den_space[i],c_len[j])
            E,v,L2 = Erk_Propogation_2D (E0,Cell_id,num_grid_x,num_grid_y,grid_size,T,dt,step,c_threhold,c_len[j],a)
            index=int(r*len(den_space)*len(c_len)+i*len(c_len)+j)
            data[0,index]=den_space[i]
            data[1,index]=c_len[j]
            data[2,index]=Find_most_distance(Cell_id)-9
            data[3,index]=v
            print(r,i,j,index/(repeat*len(den_space)*len(c_len)))
        
np.savetxt('erk_longcell_simulation.csv', data, delimiter=',')
v = np.loadtxt('erk_longcell_simulation.csv',delimiter=',')

In [ ]:
#speed vs. cell length

c_len=np.linspace(1,10,10)

c1='#EF6145'
c2='#389ADD'
c3='#45A39B'
c4='#E03960'

plt.figure(figsize=(4, 3.5))

den=1
x1data, y1data, z11data, z21data, speed = data_plot_clen_v(den,v)
plt.scatter(y1data,z21data,color=c1,s=20,alpha=0.7)
speed_error=np.std(speed,axis=0)
speed_mean=np.mean(speed,axis=0)
plt.errorbar(c_len,speed_mean,yerr=speed_error,ecolor=c1,color=c1,elinewidth=2,capsize=4,capthick=2,alpha=0.7)

den=0.8
x2data, y2data, z12data, z22data, speed = data_plot_clen_v(den,v)
plt.scatter(y2data,z22data,color=c2,s=20,alpha=0.7)
speed_error=np.std(speed,axis=0)
speed_mean=np.mean(speed,axis=0)
plt.errorbar(c_len,speed_mean,yerr=speed_error,ecolor=c2,color=c2,elinewidth=2,capsize=4,capthick=2,alpha=0.7)

den=0.5
x3data, y3data, z13data, z23data, speed = data_plot_clen_v(den,v)
plt.scatter(y3data,z23data,color=c3,s=20,alpha=0.7)
speed_error=np.std(speed,axis=0)
speed_mean=np.mean(speed,axis=0)
plt.errorbar(c_len,speed_mean,yerr=speed_error,ecolor=c3,color=c3,elinewidth=2,capsize=4,capthick=2,alpha=0.7)

In [ ]:
#speed vs. relay cell density

c1='#EF6145'
c2='#389ADD'
c3='#45A39B'
c4='#E03960'

plt.figure(figsize=(4, 3.5))

c_len=2
xdata, ydata, z1data, z2data, speed = data_plot_n(c_len,v)
speed_mean=np.mean(speed,axis=0)
plt.scatter(xdata,z2data,color=c3,s=20,alpha=0.7)
speed_error=np.std(speed,axis=0)
plt.errorbar(den_space,speed_mean,yerr=speed_error,ecolor=c3,color=c3,elinewidth=2,capsize=4,capthick=2,alpha=0.7)

c_len=5
xdata, ydata, z1data, z2data, speed = data_plot_n(c_len,v)
speed_mean=np.mean(speed,axis=0)
plt.scatter(xdata,z2data,color=c2,s=20,alpha=0.7)
speed_error=np.std(speed,axis=0)
plt.errorbar(den_space,speed_mean,yerr=speed_error,ecolor=c2,color=c2,elinewidth=2,capsize=4,capthick=2,alpha=0.7)

c_len=8
xdata, ydata, z1data, z2data, speed = data_plot_n(c_len,v)
speed_mean=np.mean(speed,axis=0)
plt.scatter(xdata,z2data,color=c1,s=20,alpha=0.7)
speed_error=np.std(speed,axis=0)
plt.errorbar(den_space,speed_mean,yerr=speed_error,ecolor=c1,color=c1,elinewidth=2,capsize=4,capthick=2,alpha=0.7)

In [ ]:
#relay cell cluster size vs. relay cell density

c1='#EF6145'
c2='#389ADD'
c3='#45A39B'

v = np.loadtxt('erk_longcell_simulation.csv',delimiter=',')

plt.figure(figsize=(4, 3.5))

c_len=4
xdata, ydata, z1data, z2data, cluster = data_plot_clus(c_len,v)
plt.scatter(xdata,z1data/90,color=c3,s=20,alpha=0.7)
speed_mean=np.mean(cluster/90,axis=0)
speed_error=np.std(cluster/90,axis=0)
#plt.errorbar(den_space,speed_mean,yerr=speed_error,ecolor=c3,color=c3,elinewidth=2,capsize=4,capthick=2,alpha=0.7)

c_len=6
xdata, ydata, z1data, z2data, cluster = data_plot_clus(c_len,v)
plt.scatter(xdata,z1data/90,color=c2,s=20,alpha=0.7)
speed_mean=np.mean(cluster/90,axis=0)
speed_error=np.std(cluster/90,axis=0)
#plt.errorbar(den_space,speed_mean,yerr=speed_error,ecolor=c2,color=c2,elinewidth=2,capsize=4,capthick=2,alpha=0.7)

c_len=8
xdata, ydata, z1data, z2data, cluster = data_plot_clus(c_len,v)
plt.scatter(xdata,z1data/90,color=c1,s=20,alpha=0.7)
speed_mean=np.mean(cluster/90,axis=0)
speed_error=np.std(cluster/90,axis=0)
#plt.errorbar(den_space,speed_mean,yerr=speed_error,ecolor=c1,color=c1,elinewidth=2,capsize=4,capthick=2,alpha=0.7)

In [ ]:
#relay cell cluster size vs. wave speed

plt.figure(figsize=(4, 3.5))

c_len=4
xdata, ydata, z1data, z2data = data_plot(c_len,v)
plt.scatter(z1data/90,z2data*60/(c_len*c_len),color=c3,s=20,alpha=0.7)

c_len=6
xdata, ydata, z1data, z2data = data_plot(c_len,v)
plt.scatter(z1data/90,z2data*60/(c_len*c_len),color=c2,s=20,alpha=0.7)

c_len=8
xdata, ydata, z1data, z2data = data_plot(c_len,v)
plt.scatter(z1data/90,z2data*60/(c_len*c_len),color=c1,s=20,alpha=0.7)